# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain_community
!pip install langchain_openai

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)


In [ ]:
prompt = ChatPromptTemplate.from_template( "Write a detailed and engaging product description for a {product}.")

In [15]:
chain = LLMChain(llm=llm, prompt=prompt)

In [18]:
product = "Smart TV"
chain.run(product=product)

"Introducing the ultimate kitchen essential - the sleek and stylish fridge that combines cutting-edge technology with unparalleled design. Say goodbye to outdated appliances and hello to the future of refrigeration.\n\nThis fridge is not just a place to store your groceries, it's a statement piece that will elevate the look of your kitchen. With its modern stainless steel finish and minimalist design, it will seamlessly blend in with any decor style.\n\nBut it's not just about looks - this fridge is packed with features that will make your life easier. The spacious interior offers ample storage space for all your food and beverages, while the adjustable shelves and door bins allow you to customize the layout to suit your needs.\n\nThe advanced cooling system ensures that your food stays fresh for longer, while the energy-efficient design helps you save on your electricity bills. And with convenient features like a water and ice dispenser, LED lighting, and a digital temperature control

## SimpleSequentialChain

In [19]:
from langchain.chains import SimpleSequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   "Write The Factory's details and product materials for a {product}."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [21]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
     "Write a detailed description of product stores and prices for a {product}."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [22]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [23]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The Factory specializes in manufacturing high-quality Smart TVs that deliver a superior viewing experience. Our Smart TVs are designed to offer seamless connectivity and advanced features for a truly immersive entertainment experience.

Product details:
- Model: Smart TV X9000
- Screen size: 55 inches
- Display type: LED
- Resolution: 4K Ultra HD
- Operating system: Android TV
- Connectivity: Wi-Fi, Bluetooth, HDMI, USB
- Smart features: Built-in voice control, streaming services (Netflix, Amazon Prime, Hulu, etc.), screen mirroring, built-in apps

Product materials:
- High-quality plastic casing
- LED display panel
- Aluminum stand
- Glass screen
- Electronic components sourced from reputable suppliers

At The Factory, we are committed to producing top-of-the-line Smart TVs that combine cutting-edge technology with sleek design aesthetics. Our rigorous quality control processes ensure that each Smart TV meets the highest standards of per

'\n\nThe Smart TV X9000 is available at The Factory store and various retailers nationwide. The price for the 55-inch model starts at $799.99, making it a competitive option for consumers looking for a high-quality Smart TV at an affordable price point.\n\nIn addition to the Smart TV X9000, The Factory also offers a range of accessories and extended warranties to enhance your viewing experience and provide peace of mind. These accessories include wall mounts, soundbars, and universal remotes, all designed to complement and maximize the functionality of your Smart TV.\n\nAt The Factory, customer satisfaction is our top priority. Our knowledgeable and friendly staff are available to assist you in choosing the right Smart TV for your needs, answering any questions you may have, and providing expert advice on setup and troubleshooting. Visit The Factory store today to experience the future of entertainment with our Smart TV X9000.'

**Repeat the above twice for different products**

## SequentialChain

In [24]:
from langchain.chains import SequentialChain

In [40]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    
  "translate reviews into Arabic:\n {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translate" #Give a name to your output
                    )


In [42]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "summarize reviews into English:\n {review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [38]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
      "translate reviews into Spanish:\n {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [37]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(


        "occording to the review:\n {review}\n\n write a professional follow-up message to the customer."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup"
                     )


In [48]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translate","language","summary", "followup"],
    verbose=True
)

In [49]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translate': 'أجد الطعم سيء. الرغوة لا تدوم، هذا غريب. أشتري نفس المنتجات من المتاجر والطعم أفضل بكثير...\nهل هو دفعة قديمة أم تقليدية؟',
 'language': 'Encuentro el sabor mediocre. La espuma no se sostiene, es extraño. Compro los mismos en el comercio y el sabor es mucho mejor... ¿Lote viejo o falsificación!?',
 'summary': "The taste is poor and the foam doesn't last. I buy the same ones at the store and the taste is much better. Is it an old batch or a counterfeit?",
 'followup': 'Dear valued customer,\n\nThank you for bringing this issue to our attention. We are sorry to hear that you found the taste of our product to be subpar and that the foam did not hold as expected. We take quality control very seriously and would like to assure you that our products are not counterfeit.\n\nWe would like to investigate 

**Repeat the above twice for different products or reviews**

## Router Chain

In [55]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [56]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [52]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [53]:
llm = ChatOpenAI(temperature=0)

In [54]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [60]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\riina\AppData\Local\Temp\ipykernel_9400\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [61]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [62]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [63]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**